# 1. Load packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd## 1.1 Define output folder

output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2'
output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

36172
Use existing analysis folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\analysis.
Use existing figure folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\figures.


In [2]:
import seaborn as sns
import scanpy as sc
import numpy as np

## 1.1 Define output folder

In [3]:
output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables'

# 2. Load primary probes and primers

In [6]:
#probe_folder = os.path.join(output_main_folder, 'Primary')
probe_folder = output_main_folder
probe_fname = os.path.join(probe_folder, r'Table_S1_MOp_primary_oligopropes.xlsx')

probe_df_dict = {'RNA MERFISH probes':None,
                 'Genome chromatin probes':None, 
                 'Marker chromatin probes':None, 
                 'Super-enhancer chromatin probes':None, 
                 }
for sheet_name in probe_df_dict.keys():
    probe_df = pd.read_excel(probe_fname, sheet_name = sheet_name)
    probe_df_dict[sheet_name] = probe_df

In [67]:
primer_fname =  os.path.join(probe_folder, r'Table_S3_MOp_primer_and_readout_oligopropes.xlsx')
primer_df = pd.read_excel(primer_fname, sheet_name = 'Primers')
primer_df

,Library,Sublibrary,Forward primer name,Reverse primer name,Forward primer seq,Reverse primer seq
0,Genome chromatin,Genome chromatin_set_1_and_2,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
1,Genome chromatin,Genome chromatin_set_3,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
2,Genome chromatin,Genome chromatin_set_4,W1B05_primer_16,W1B04_primer_15,CGGGTTTCGTTGCGCACACC,TAATACGACTCACTATAGGGCTTGTGCATCGCGCCAAAGA
3,Marker chromatin,Marker chromatin_set_all,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
4,Super-enhancer chromatin,Super-enhancer chromatin_set_all,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
5,RNA MERFISH,RNA MERFISH_set_all,MOp_primer_F,MOp_primer_R,CGCGGTGGTGGTACGACATG,TAATACGACTCACTATAGTTGTCCGTCCTAGTTGCGCT


In [96]:
T7_seq = r'TAATACGACTCACTATAG'

## RNA MERFISH

In [76]:
sheet_name = 'RNA MERFISH probes'
probe_df = probe_df_dict[sheet_name]
probe_df['length'] = probe_df['sequence'].apply(lambda x:len(x))
probe_df
print(np.unique(probe_df['length'], return_counts=True))
print(len(probe_df['length']))

(array([112], dtype=int64), array([22005], dtype=int64))
22005


In [97]:
print(len(primer_df[primer_df['Library'] == sheet_name.split(' probes')[0]]['Forward primer seq'].values[0]))
print(len(primer_df[primer_df['Library'] == sheet_name.split(' probes')[0]]['Reverse primer seq'].values[0])-len(T7_seq))

20
20


In [98]:
probe_seq_list = probe_df['sequence'].tolist()
probe_seq_list[:3]

['CGCGGTGGTGGTACGACATGAAGAGTGAGTAGTAGTGGAGTGTTAAGAACTGGGGAGCTGGAAAATGAGTGAAGGGTGTGTTTGTAAAGGGTAGCGCAACTAGGACGGACAA',
 'CGCGGTGGTGGTACGACATGAAGGAGGAGGGTAATGATAGAGGAGGCTTCTCTTTCTAGTACATTTCTGCAAAGAGTGAGTAGTAGTGGAGTAGCGCAACTAGGACGGACAA',
 'CGCGGTGGTGGTACGACATGAAGAGTGAGTAGTAGTGGAGTTTTTGGAGTTCAGGCTGTGGGTACAGACCTATGGGAGGATTGAGATGAGTTAGCGCAACTAGGACGGACAA']

In [99]:
probe_seq_list[-3:]

['CGCGGTGGTGGTACGACATGAATAGGAAATGGTGGTAGTGTTTCTTCCTTCATAAGGTCTGCATCATTGCTAAGGAGGAGGGTAATGATAGAAGCGCAACTAGGACGGACAA',
 'CGCGGTGGTGGTACGACATGATGGGAGGATTGAGATGAGTTGCAGCACTGGTTGGATTAAAGGATTCAAGTAATAGGAAATGGTGGTAGTGTAGCGCAACTAGGACGGACAA',
 'CGCGGTGGTGGTACGACATGAATAGGAAATGGTGGTAGTGTTCAAGTGTACTGCCATATTTAAAGCCTCCTAGAGATTAGAGATGAGTTGGAAGCGCAACTAGGACGGACAA']

In [100]:
T7_seq = r'TAATACGACTCACTATAG'
forward_primer = probe_seq_list[0][:20]
reverse_primer_rc = probe_seq_list[0][-20:]
readout_seq_1 = probe_seq_list[0][21:41]
readout_seq_2 = probe_seq_list[0][72:-20]
target_seq = probe_seq_list[0][41:71]
linker = 'A'

In [101]:
print(readout_seq_1)
print(readout_seq_2)

AGAGTGAGTAGTAGTGGAGT
AGGGTGTGTTTGTAAAGGGT


In [102]:
forward_primer + linker + readout_seq_1 + target_seq + linker + readout_seq_2 + reverse_primer_rc

'CGCGGTGGTGGTACGACATGAAGAGTGAGTAGTAGTGGAGTGTTAAGAACTGGGGAGCTGGAAAATGAGTGAAGGGTGTGTTTGTAAAGGGTAGCGCAACTAGGACGGACAA'

## Genome chromatin

In [103]:
sheet_name = 'Genome chromatin probes'
probe_df = probe_df_dict[sheet_name]
probe_df['length'] = probe_df['sequence'].apply(lambda x:len(x))
probe_df
print(np.unique(probe_df['length'], return_counts=True))
print(len(probe_df['length']))

(array([142], dtype=int64), array([173891], dtype=int64))
173891


In [120]:
sel_primer_df = primer_df[primer_df['Library'] == sheet_name.split(' probes')[0]]
for _seq in sel_primer_df['Forward primer seq'].tolist():
    print (len(_seq))
                    
for _seq in sel_primer_df['Reverse primer seq'].tolist():
    print (T7_seq, _seq[len(T7_seq):], len(_seq)-len(T7_seq))

20
20
20
TAATACGACTCACTATAG GGATTGCCGCATGGTTTCCG 20
TAATACGACTCACTATAG GGCCATTGCCCGCGAGGTCGAG 22
TAATACGACTCACTATAG GGCTTGTGCATCGCGCCAAAGA 22


In [118]:
from Bio.Seq import Seq
reverse_primer_rc = str(Seq('GGATTGCCGCATGGTTTCCG').reverse_complement())
probe_df[probe_df['sequence'].str.contains(reverse_primer_rc)]

,name,sequence,length
0,loc_1:3740000-3760000_gene_1_pb_23_pos_2742_st...,CCCGCAATGGCTGACAACCGTATACATTAGGATTGCGAGCTGTCAT...,142
1,loc_1:3740000-3760000_gene_1_pb_24_pos_2784_st...,CCCGCAATGGCTGACAACCGTATACATTAGGATTGCGAGCAATAGA...,142
2,loc_1:3740000-3760000_gene_1_pb_25_pos_2905_st...,CCCGCAATGGCTGACAACCGTATACATTAGGATTGCGAGCATATAA...,142
3,loc_1:3740000-3760000_gene_1_pb_26_pos_2980_st...,CCCGCAATGGCTGACAACCGTATACATTAGGATTGCGAGCACTCAC...,142
4,loc_1:3740000-3760000_gene_1_pb_27_pos_3017_st...,CCCGCAATGGCTGACAACCGTATACATTAGGATTGCGAGCTTCAAA...,142
...,...,...,...
57931,loc_Y:1240000-1260000_gene_1062_pb_163_pos_196...,CCCGCAATGGCTGACAACCGTACCGCGGCTTAGTAGTTGAGTGCAA...,142
57932,loc_Y:1240000-1260000_gene_1062_pb_164_pos_197...,CCCGCAATGGCTGACAACCGTACCGCGGCTTAGTAGTTGATATAGA...,142
57933,loc_Y:1240000-1260000_gene_1062_pb_165_pos_197...,CCCGCAATGGCTGACAACCGTACCGCGGCTTAGTAGTTGATAAAAT...,142
57934,loc_Y:1240000-1260000_gene_1062_pb_166_pos_198...,CCCGCAATGGCTGACAACCGTACCGCGGCTTAGTAGTTGATGAGGT...,142


In [122]:
from Bio.Seq import Seq
reverse_primer_rc = str(Seq('CCATTGCCCGCGAGGTCGAG').reverse_complement())
probe_df[probe_df['sequence'].str.contains(reverse_primer_rc)]

# therefore two additional N were added in between T7 and reverse_primer_rc for the primers that were 22 nts.
# this will not affect the primary probe composition

,name,sequence,length
57936,loc_1:6240000-6260000_gene_2_pb_58_pos_5958_st...,CGCAAACTGGTGCGGAAGGCCTTAGTCTCGCGCGCCGGATTTTCCC...,142
57937,loc_1:6240000-6260000_gene_2_pb_59_pos_6065_st...,CGCAAACTGGTGCGGAAGGCCTTAGTCTCGCGCGCCGGATCTTTCC...,142
57938,loc_1:6240000-6260000_gene_2_pb_60_pos_6208_st...,CGCAAACTGGTGCGGAAGGCCTTAGTCTCGCGCGCCGGATAGTGTT...,142
57939,loc_1:6240000-6260000_gene_2_pb_61_pos_6245_st...,CGCAAACTGGTGCGGAAGGCCTTAGTCTCGCGCGCCGGATGATGTA...,142
57940,loc_1:6240000-6260000_gene_2_pb_62_pos_6335_st...,CGCAAACTGGTGCGGAAGGCCTTAGTCTCGCGCGCCGGATGTGGAG...,142
...,...,...,...
86968,loc_X:163740000-163760000_gene_1058_pb_252_pos...,CGCAAACTGGTGCGGAAGGCACGCGCTTATTGCCGAACATCTGTAT...,142
86969,loc_X:163740000-163760000_gene_1058_pb_253_pos...,CGCAAACTGGTGCGGAAGGCACGCGCTTATTGCCGAACATGAGAGT...,142
86970,loc_X:163740000-163760000_gene_1058_pb_254_pos...,CGCAAACTGGTGCGGAAGGCACGCGCTTATTGCCGAACATGTAATT...,142
86971,loc_X:163740000-163760000_gene_1058_pb_255_pos...,CGCAAACTGGTGCGGAAGGCACGCGCTTATTGCCGAACATGGTTTG...,142


In [37]:
probe_seq_list[-3:]

['CGGCTCGCAGCGTGTAAACGACGGGTGGATTGCAGCTTACGAGATTAGAGACTTTAAAGGCTACTGCAGTCCAAGAGCATGTTAACAGGTAATAAGGCGAGCGATACACCCTGGACGCAAATCCGCCGGTGATCCTGAAATG',
 'CGGCTCGCAGCGTGTAAACGACGGGTGGATTGCAGCTTACGGAATCATCTGCCAAGGTCTGCACCTCTGGGAAAAATTGAGATAACAGGTAATAAGGCGAGCGATACACCCTGGACGCAAATCCGCCGGTGATCCTGAAATG',
 'CGGCTCGCAGCGTGTAAACGACGGGTGGATTGCAGCTTACGAAGGGAGGGAAGGTATTCTCTGATGGTGAAAGGTGAATGGATAACAGGTAATAAGGCGAGCGATACACCCTGGACGCAAATCCGCCGGTGATCCTGAAATG']

In [41]:
T7_seq = r'TAATACGACTCACTATAG'
forward_primer = probe_seq_list[0][:20]
reverse_primer_rc = probe_seq_list[0][-20:]
readout_seq_1 = probe_seq_list[0][20:40]
readout_seq_2 = probe_seq_list[0][82:102]
readout_seq_3 = probe_seq_list[0][102:122]
target_seq = probe_seq_list[0][40:82]
linker = 'A'

In [43]:
print(forward_primer)
print(reverse_primer_rc)

CCCGCAATGGCTGACAACCG
CGGAAACCATGCGGCAATCC


In [42]:
print(readout_seq_1)
print(readout_seq_2)
print(readout_seq_3)

TATACATTAGGATTGCGAGC
TACTTATTGCCGGCCACGAT
TCGCATCCACGAACCAGGTC


In [44]:
forward_primer + readout_seq_1 + target_seq + readout_seq_2 + readout_seq_3 + reverse_primer_rc

'CCCGCAATGGCTGACAACCGTATACATTAGGATTGCGAGCTGTCATAACTGTGTGAGGACCTCTCCCTTAACTCATTCCACTTACTTATTGCCGGCCACGATTCGCATCCACGAACCAGGTCCGGAAACCATGCGGCAATCC'

## Super-enhancer chromatin

In [123]:
sheet_name = 'Super-enhancer chromatin probes'
probe_df = probe_df_dict[sheet_name]
probe_df['length'] = probe_df['sequence'].apply(lambda x:len(x))
probe_df
print(np.unique(probe_df['length'], return_counts=True))
print(len(probe_df['length']))

(array([142], dtype=int64), array([115776], dtype=int64))
115776


In [124]:
sel_primer_df = primer_df[primer_df['Library'] == sheet_name.split(' probes')[0]]
for _seq in sel_primer_df['Forward primer seq'].tolist():
    print (len(_seq))
                    
for _seq in sel_primer_df['Reverse primer seq'].tolist():
    print (T7_seq, _seq[len(T7_seq):], len(_seq)-len(T7_seq))

20
TAATACGACTCACTATAG GGATTGCCGCATGGTTTCCG 20


In [47]:
probe_seq_list[-3:]

['CCCGCAATGGCTGACAACCGTGTACCAGTACGCAGCCGACCGGACATGGAGGACGCGCTTCCAGACACCGGGGGACGGACTCGGGTACTTACGCCGCGGTATCGGGTACACCGTCATCACACCGGAAACCATGCGGCAATCC',
 'CCCGCAATGGCTGACAACCGTGTACCAGTACGCAGCCGACTCCCGGAGACGGTGGCGGCACGCAGCGGCCACTCACAGTCATGGGTACTTACGCCGCGGTATCGGGTACACCGTCATCACACCGGAAACCATGCGGCAATCC',
 'CCCGCAATGGCTGACAACCGTGTACCAGTACGCAGCCGACCCCCGCCTCACACAGTTCCGAGTTTGGGGGTGTTTGGCTTTGGGGTACTTACGCCGCGGTATCGGGTACACCGTCATCACACCGGAAACCATGCGGCAATCC']

In [48]:
T7_seq = r'TAATACGACTCACTATAG'
forward_primer = probe_seq_list[0][:20]
reverse_primer_rc = probe_seq_list[0][-20:]
readout_seq_1 = probe_seq_list[0][20:40]
readout_seq_2 = probe_seq_list[0][82:102]
readout_seq_3 = probe_seq_list[0][102:122]
target_seq = probe_seq_list[0][40:82]
linker = 'A'

In [49]:
print(forward_primer)
print(reverse_primer_rc)

CCCGCAATGGCTGACAACCG
CGGAAACCATGCGGCAATCC


In [50]:
print(readout_seq_1)
print(readout_seq_2)
print(readout_seq_3)

TAGATAGAGACGTTCGTGAA
GATTTATCTATGACCGGGAT
GACAGGACGTACCGCTTACC


In [51]:
forward_primer + readout_seq_1 + target_seq + readout_seq_2 + readout_seq_3 + reverse_primer_rc

'CCCGCAATGGCTGACAACCGTAGATAGAGACGTTCGTGAACTCCCTATTGACCTCACTCTATGGTACATCCTCTGTTATGACGATTTATCTATGACCGGGATGACAGGACGTACCGCTTACCCGGAAACCATGCGGCAATCC'

## Marker chromatin

In [125]:
sheet_name = 'Marker chromatin probes'
probe_df = probe_df_dict[sheet_name]
probe_df['length'] = probe_df['sequence'].apply(lambda x:len(x))
probe_df
print(np.unique(probe_df['length'], return_counts=True))
print(len(probe_df['length']))

(array([120], dtype=int64), array([3360], dtype=int64))
3360


In [126]:
sel_primer_df = primer_df[primer_df['Library'] == sheet_name.split(' probes')[0]]
for _seq in sel_primer_df['Forward primer seq'].tolist():
    print (len(_seq))
                    
for _seq in sel_primer_df['Reverse primer seq'].tolist():
    print (T7_seq, _seq[len(T7_seq):], len(_seq)-len(T7_seq))

20
TAATACGACTCACTATAG GGCCATTGCCCGCGAGGTCGAG 22


In [55]:
len(probe_seq_list[:3][0])

120

In [137]:
probe_seq_list = probe_df['sequence'].tolist()
probe_seq_list[:3]

['CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGTTCCATTCATAGGGCTCTAGCGTCAACTGCTCTTGGGGCTTGCCGTTCGTCTGCGCTTGGCTCGACCTCGCGGGCAATGG',
 'CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGCAGCCTCGGCAGAAGCTGTGCTTGCGTCTGAAGTGAAGAGTGCCGTTCGTCTGCGCTTGGCTCGACCTCGCGGGCAATGG',
 'CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGACATCTTCCTGCAGAATGGAGGATACTTTGTGATTTCCATTGCCGTTCGTCTGCGCTTGGCTCGACCTCGCGGGCAATGG']

In [138]:
probe_seq_list[-3:]

['CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAGAGAGATGGGAGGGCATGAATAGAAAGAACATGATGCCGGAAGTCCGCGCAGGAACGTGACTCGACCTCGCGGGCAATGG',
 'CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAAGCAGAAAAGAACATGACATTTAACTCCTCCCTCACAGTAAAGTCCGCGCAGGAACGTGACTCGACCTCGCGGGCAATGG',
 'CGCAAACTGGTGCGGAAGGCAAGTCCGCGCAGGAACGTGAAGCAGCTCACAGAAGTTGATGCTGTAAATGCAAATGTCTTAAGTCCGCGCAGGAACGTGACTCGACCTCGCGGGCAATGG']

In [139]:
T7_seq = r'TAATACGACTCACTATAG'
forward_primer = probe_seq_list[0][:20]
reverse_primer_rc = probe_seq_list[0][-20:]
readout_seq_1 = probe_seq_list[0][20:40]
readout_seq_2 = probe_seq_list[0][80:100]

target_seq = probe_seq_list[0][40:80]
linker = 'A'

In [140]:
print(forward_primer)
print(reverse_primer_rc)

CGCAAACTGGTGCGGAAGGC
CTCGACCTCGCGGGCAATGG


In [141]:
print(readout_seq_1)
print(readout_seq_2)

TGCCGTTCGTCTGCGCTTGG
TGCCGTTCGTCTGCGCTTGG


In [142]:
forward_primer + readout_seq_1 + target_seq + readout_seq_2 + reverse_primer_rc

'CGCAAACTGGTGCGGAAGGCTGCCGTTCGTCTGCGCTTGGTTCCATTCATAGGGCTCTAGCGTCAACTGCTCTTGGGGCTTGCCGTTCGTCTGCGCTTGGCTCGACCTCGCGGGCAATGG'